In [ ]:
#! /usr/bin/env python3

"""
IPseek - A Python-based Network Scanner
Author: Niyumi Weerathunga
Description: Scans a subnet for live hosts and open ports using SYN scan (Scapy).
"""

import logging
import sys
import os
import re
import argparse
from scapy.all import sr, sr1, send, hexdump
from scapy.layers.inet import IP, ICMP, TCP

logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

# Prevent the OS from sending RST packets that would interfere with our scan
os.system('iptables -A OUTPUT -p tcp --tcp-flags RST RST -j DROP')

def find_position_of_separator(ip, pos):
    index = -1
    for _ in range(pos):
        index = ip.find(".", index + 1)
    return index

def increment_octet(ip_part, octet_num):
    if octet_num != 0:
        idx = find_position_of_separator(ip_part, octet_num - 1)
        octet = int(ip_part[idx + 1:])
        if octet == 255:
            return increment_octet(ip_part[:idx - 2], octet_num - 1)
        return ip_part[:idx + 1] + str(octet + 1)
    return ip_part

def print_usage():
    print("""
Usage Error: Invalid arguments.

Correct format:
  sudo python3 IPseek.py -pSTART-END SUBNET

Example:
  sudo python3 IPseek.py -p20-25 192.168.1.0/30
    """)
    sys.exit(1)

def parse_args():
    parser = argparse.ArgumentParser(description="IPseek - Network and Port Scanner")
    parser.add_argument("-p", dest="port_range", help="Port range (e.g., -p20-80)")
    parser.add_argument("subnet", help="Target subnet (e.g., 192.168.1.0/30)")
    # Fix for notebook/IDE environments
    if "__file__" in globals():
        return parser.parse_args()
    else:
        return parser.parse_args(args=["-p20-25", "192.168.1.0/30"])  # Default for testing

def generate_ip_list(subnet):
    if "/" not in subnet:
        return [subnet]

    ip_base, mask = subnet.split("/")
    mask = int(mask)
    total_ips = 2 ** (32 - mask)
    pos = find_position_of_separator(ip_base, 3)
    base = ip_base[:pos + 1]
    last = int(ip_base[pos + 1:])

    ips = []
    for _ in range(1, total_ips - 1):
        last += 1
        if last > 255:
            last = 0
            base = increment_octet(base[:-1], 3) + "."
        ips.append(base + str(last))
    return ips

def scan_ports(ip_list, port_list):
    open_ports = []
    for ip in ip_list:
        print(f"[*] Scanning host: {ip}")
        response = sr1(IP(dst=ip)/ICMP(), timeout=2, verbose=0)
        if response:
            print(f"    [✔] Host is up")
            for port in port_list:
                pkt = IP(dst=ip)/TCP(dport=port, flags="S")
                ans = sr1(pkt, timeout=2, verbose=0)
                if ans and ans.haslayer(TCP) and ans.getlayer(TCP).flags == 18:
                    print(f"    [+] Port {port} is OPEN")
                    open_ports.append((ip, port))
        else:
            print(f"    [✘] Host did not respond")
    return open_ports

def probe_services(open_ports):
    print("\n[**] Probing services...")
    for ip, port in open_ports:
        pkt_ip = IP(dst=ip)
        syn = TCP(dport=port, flags='S')
        syn_ack = sr1(pkt_ip / syn, timeout=2, verbose=0)

        if not syn_ack:
            print(f"[!] No response from {ip}:{port}")
            continue

        ack = TCP(dport=port, flags='A', seq=syn_ack.ack, ack=syn_ack.seq + 1)
        try:
            http_request, _ = sr(pkt_ip / ack / "GET / HTTP/1.0\r\n\r\n", timeout=2, verbose=0, multi=True)
            load = http_request[1][1].getlayer(TCP).load
            print(f"\n[+] {ip}:{port} replied:")
            print(hexdump(load[:1024]))
        except:
            print(f"[!] No useful response from {ip}:{port}")

        # Close connection
        send(pkt_ip / TCP(dport=port, flags='RA', seq=syn_ack.ack, ack=syn_ack.seq + 1), verbose=0)

def main():
    args = parse_args()

    # Validate port range format
    port_match = re.match(r"(\d+)-(\d+)", args.port_range)
    if not port_match:
        print_usage()

    start_port, end_port = map(int, port_match.groups())
    if start_port > end_port:
        print_usage()

    port_list = list(range(start_port, end_port + 1))
    ip_list = generate_ip_list(args.subnet)

    print(f"\n[IPseek] Scanning {len(ip_list)} host(s) on subnet {args.subnet}...")
    open_ports = scan_ports(ip_list, port_list)
    print(f"\n[IPseek] Scan complete. {len(open_ports)} open ports found.")

    if open_ports:
        probe_services(open_ports)

    os.system('iptables -D OUTPUT -p tcp --tcp-flags RST RST -j DROP')
    print("\n[IPseek] Done.")

if __name__ == '__main__':
    main()


[IPseek] Scanning 2 host(s) on subnet 192.168.1.0/30...
[*] Scanning host: 192.168.1.1
    [✘] Host did not respond
[*] Scanning host: 192.168.1.2
    [✘] Host did not respond

[IPseek] Scan complete. 0 open ports found.

[IPseek] Done.


In [4]:
#! /usr/bin/env python3

"""
IPseek - A Python-based Network Scanner
Author: Your Name
Description: Scans a subnet for live hosts and open ports using SYN scan (Scapy).
"""

import logging
import sys
import os
import re
import argparse
from scapy.all import sr, sr1, send, hexdump
from scapy.layers.inet import IP, ICMP, TCP

logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

# Prevent the OS from sending RST packets that would interfere with our scan
os.system('iptables -A OUTPUT -p tcp --tcp-flags RST RST -j DROP')

def find_position_of_separator(ip, pos):
    index = -1
    for _ in range(pos):
        index = ip.find(".", index + 1)
    return index

def increment_octet(ip_part, octet_num):
    if octet_num != 0:
        idx = find_position_of_separator(ip_part, octet_num - 1)
        octet = int(ip_part[idx + 1:])
        if octet == 255:
            return increment_octet(ip_part[:idx - 2], octet_num - 1)
        return ip_part[:idx + 1] + str(octet + 1)
    return ip_part

def print_usage():
    print("""
Usage Error: Invalid arguments.

Correct format:
  sudo python3 IPseek.py -pSTART-END SUBNET

Example:
  sudo python3 IPseek.py -p20-25 192.168.56.0/30
    """)
    sys.exit(1)

def parse_args():
    parser = argparse.ArgumentParser(description="IPseek - Network and Port Scanner")
    parser.add_argument("-p", dest="port_range", help="Port range (e.g., -p20-80)")
    parser.add_argument("subnet", help="Target subnet (e.g., 192.168.56.0/30)")
    # Fix for notebook/IDE environments
    if "__file__" in globals():
        return parser.parse_args()
    else:
        return parser.parse_args(args=["-p20-80", "192.168.56.0/30"])  # Default for your IP

def generate_ip_list(subnet):
    if "/" not in subnet:
        return [subnet]

    ip_base, mask = subnet.split("/")
    mask = int(mask)
    total_ips = 2 ** (32 - mask)
    pos = find_position_of_separator(ip_base, 3)
    base = ip_base[:pos + 1]
    last = int(ip_base[pos + 1:])

    ips = []
    for _ in range(1, total_ips - 1):
        last += 1
        if last > 255:
            last = 0
            base = increment_octet(base[:-1], 3) + "."
        ips.append(base + str(last))
    return ips

def scan_ports(ip_list, port_list):
    open_ports = []
    for ip in ip_list:
        print(f"[*] Scanning host: {ip}")
        response = sr1(IP(dst=ip)/ICMP(), timeout=2, verbose=0)
        if response:
            print(f"    [✔] Host is up")
            for port in port_list:
                pkt = IP(dst=ip)/TCP(dport=port, flags="S")
                ans = sr1(pkt, timeout=2, verbose=0)
                if ans and ans.haslayer(TCP) and ans.getlayer(TCP).flags == 18:
                    print(f"    [+] Port {port} is OPEN")
                    open_ports.append((ip, port))
        else:
            print(f"    [✘] Host did not respond")
    return open_ports

def probe_services(open_ports):
    print("\n[**] Probing services...")
    for ip, port in open_ports:
        pkt_ip = IP(dst=ip)
        syn = TCP(dport=port, flags='S')
        syn_ack = sr1(pkt_ip / syn, timeout=2, verbose=0)

        if not syn_ack:
            print(f"[!] No response from {ip}:{port}")
            continue

        ack = TCP(dport=port, flags='A', seq=syn_ack.ack, ack=syn_ack.seq + 1)
        try:
            http_request, _ = sr(pkt_ip / ack / "GET / HTTP/1.0\r\n\r\n", timeout=2, verbose=0, multi=True)
            load = http_request[1][1].getlayer(TCP).load
            print(f"\n[+] {ip}:{port} replied:")
            print(hexdump(load[:1024]))
        except:
            print(f"[!] No useful response from {ip}:{port}")

        # Close connection
        send(pkt_ip / TCP(dport=port, flags='RA', seq=syn_ack.ack, ack=syn_ack.seq + 1), verbose=0)

def main():
    args = parse_args()

    # Validate port range format
    port_match = re.match(r"(\d+)-(\d+)", args.port_range)
    if not port_match:
        print_usage()

    start_port, end_port = map(int, port_match.groups())
    if start_port > end_port:
        print_usage()

    port_list = list(range(start_port, end_port + 1))
    ip_list = generate_ip_list(args.subnet)

    print(f"\n[IPseek] Scanning {len(ip_list)} host(s) on subnet {args.subnet}...")
    open_ports = scan_ports(ip_list, port_list)
    print(f"\n[IPseek] Scan complete. {len(open_ports)} open ports found.")

    if open_ports:
        probe_services(open_ports)

    os.system('iptables -D OUTPUT -p tcp --tcp-flags RST RST -j DROP')
    print("\n[IPseek] Done.")

if __name__ == '__main__':
    main()


[IPseek] Scanning 2 host(s) on subnet 192.168.56.0/30...
[*] Scanning host: 192.168.56.1
    [✔] Host is up
[*] Scanning host: 192.168.56.2
    [✘] Host did not respond

[IPseek] Scan complete. 0 open ports found.

[IPseek] Done.
